#Build an Organizational Hierarchy with a simple employee census

Given a standard employee census flat file containing employee ID, name, role, and manager, I wanted to build a quick way to arrange employees into an organizational hierarchy showing full reporting lines. As such, I built a function that takes a census flat file (as a dataframe) and the ID of the "top employee" i.e. whomever sits at the top of the organization strucutre - in this case the CEO. 

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

In [2]:
# mount google drive to access dummy data saved there
drive.mount('/content/drive')

Mounted at /content/drive


In [119]:
# load dummy data into dataframe - fill any blanks with 'XX'
df_path = '/content/drive/My Drive/colabData/DummyCensus.xlsx'
df = pd.read_excel(df_path)
df = df.fillna('XX')
df.head(25)

,EmpID,EmpName,EmpTitle,EmpReportsTo
0,Emp001,"Jones, Jim",CEO,XX
1,Emp002,"Barnes, Bob","SVP, Product",Emp026
2,Emp003,"Carlton, Hans","Manager, Marketing",Emp022
3,Emp004,"Seeler, Chad","Analyst, Marketing",Emp003
4,Emp005,"Bobthorton, Billy","Analyst, Marketing",Emp025
5,Emp006,"Dolf, A. Hit",Ombudsman,Emp001
6,Emp007,"Irons, Jeremy","VP, Finance",Emp013
7,Emp008,"Barnes, Bob D.","VP, Accounting",Emp013
8,Emp009,"Bruce, Lenny","SVP, Development",Emp001
9,Emp010,"McGee, Jenny","Manager, Finance",Emp007


In [143]:
def buildOrg(df, top):
  
  # define varibles for iteration
  i = 1
  check = False
  r_suffix = '_' + str(i)

  if i - 1 == 0:
    l_key = 'EmpID'
  else:
    l_key = 'EmpID_' + str(i-1)
  
  sort_list = ['EmpID']

  # match all employees reporting to top of hierarchy
  df_lookup = df[df['EmpID']==top]

  # match to employee downline as long as the resulting set is not empty
  while check == False:

    if i - 1 == 0:
      l_key = 'EmpID'
    else:
      l_key = 'EmpID_' + str(i-1)
    
    df_lookup = df_lookup.merge(df, how='left', left_on=l_key, right_on='EmpReportsTo', suffixes=('', r_suffix))
    check = df_lookup[l_key].isnull().all()
    
    # increment
    i = i + 1
    r_suffix = '_' + str(i)
    sort_list.append(l_key)
  
  # clean up the resulting dataframe and output
  df_new = df_lookup

  for (columnName, columnData) in df_lookup.iteritems():
    if 'EmpReportsTo' in str(columnName):
      df_new = df_new.drop(columnName, axis=1)
    elif df_lookup[columnName].isnull().all():
      df_new = df_new.drop(columnName, axis=1)
  
  # sort dataframe
  sort_list = sort_list[:-1]
  sort_list.reverse()
  df_new = df_new.sort_values(by=sort_list, ascending=False)

  return df_new

In [148]:
# store the output 
df_org = buildOrg(df, 'Emp001')
df_org.head(10)

,EmpID,EmpName,EmpTitle,EmpID_1,EmpName_1,EmpTitle_1,EmpID_2,EmpName_2,EmpTitle_2,EmpID_3,EmpName_3,EmpTitle_3,EmpID_4,EmpName_4,EmpTitle_4
2,Emp001,"Jones, Jim",CEO,Emp013,"Foxtrot, Liz",CFO,Emp007,"Irons, Jeremy","VP, Finance",Emp010,"McGee, Jenny","Manager, Finance",Emp023,"Marvel, Captain","Sr. Analyst, Finance"
4,Emp001,"Jones, Jim",CEO,Emp013,"Foxtrot, Liz",CFO,Emp008,"Barnes, Bob D.","VP, Accounting",Emp011,"Goena, Jack","Manager, Accounting",Emp021,"Doe, Jane J.",Staff Accountant II
3,Emp001,"Jones, Jim",CEO,Emp013,"Foxtrot, Liz",CFO,Emp008,"Barnes, Bob D.","VP, Accounting",Emp011,"Goena, Jack","Manager, Accounting",Emp020,"Keytar, Lo",Staff Accountant I
5,Emp001,"Jones, Jim",CEO,Emp013,"Foxtrot, Liz",CFO,Emp017,"Diaz, Cameron","SVP, Operations",Emp018,"Pitt, Jennifer","Manager, Operations",Emp019,"Harris, Kamalo","Analyst, Operations"
9,Emp001,"Jones, Jim",CEO,Emp026,"Dingleberry, Xiao",CTO,Emp002,"Barnes, Bob","SVP, Product",Emp016,"Duff, Hilary","Manager, Product",Emp012,"Lockhart, Cal","Analyst, Product"
8,Emp001,"Jones, Jim",CEO,Emp022,"Who, Doktor",CMO,Emp025,"Pup, Jimbo","Manager, Marketing",Emp005,"Bobthorton, Billy","Analyst, Marketing",NaN,NaN,NaN
7,Emp001,"Jones, Jim",CEO,Emp022,"Who, Doktor",CMO,Emp003,"Carlton, Hans","Manager, Marketing",Emp004,"Seeler, Chad","Analyst, Marketing",NaN,NaN,NaN
1,Emp001,"Jones, Jim",CEO,Emp009,"Bruce, Lenny","SVP, Development",Emp024,"Danes, Stevie","Manager, Development",NaN,NaN,NaN,NaN,NaN,NaN
0,Emp001,"Jones, Jim",CEO,Emp006,"Dolf, A. Hit",Ombudsman,Emp015,"Gross, Terry",Legal Counsel,NaN,NaN,NaN,NaN,NaN,NaN
6,Emp001,"Jones, Jim",CEO,Emp014,"Joe, G.I.",Chief of Staff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
#transpose the output
df_t = df_org.transpose()
df_t.head(25)

,2,4,3,5,9,8,7,1,0,6
EmpID,Emp001,Emp001,Emp001,Emp001,Emp001,Emp001,Emp001,Emp001,Emp001,Emp001
EmpName,"Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim","Jones, Jim"
EmpTitle,CEO,CEO,CEO,CEO,CEO,CEO,CEO,CEO,CEO,CEO
EmpID_1,Emp013,Emp013,Emp013,Emp013,Emp026,Emp022,Emp022,Emp009,Emp006,Emp014
EmpName_1,"Foxtrot, Liz","Foxtrot, Liz","Foxtrot, Liz","Foxtrot, Liz","Dingleberry, Xiao","Who, Doktor","Who, Doktor","Bruce, Lenny","Dolf, A. Hit","Joe, G.I."
EmpTitle_1,CFO,CFO,CFO,CFO,CTO,CMO,CMO,"SVP, Development",Ombudsman,Chief of Staff
EmpID_2,Emp007,Emp008,Emp008,Emp017,Emp002,Emp025,Emp003,Emp024,Emp015,NaN
EmpName_2,"Irons, Jeremy","Barnes, Bob D.","Barnes, Bob D.","Diaz, Cameron","Barnes, Bob","Pup, Jimbo","Carlton, Hans","Danes, Stevie","Gross, Terry",NaN
EmpTitle_2,"VP, Finance","VP, Accounting","VP, Accounting","SVP, Operations","SVP, Product","Manager, Marketing","Manager, Marketing","Manager, Development",Legal Counsel,NaN
EmpID_3,Emp010,Emp011,Emp011,Emp018,Emp016,Emp005,Emp004,NaN,NaN,NaN
